In [47]:
import json
import os
import pandas as pd
import pyshark
import random
import re
from collections import OrderedDict
from json import JSONDecoder
from json import loads
from json import dumps
from os.path import isfile, join
from os import listdir
from pprint import pprint
from subprocess import Popen, PIPE, check_output

# Extract funtion is used to flatten nested dictionaries to a Lvl 1 Dictionary
# Cons: It does not avoid collisions
def extract(dict_in, dict_out):
    for key, value in dict_in.iteritems():
        if isinstance(value, dict): # If value itself is dictionary
            extract(value, dict_out)
        elif isinstance(value, unicode):
            # Write to dict_out
            dict_out[key] = value
    return dict_out

# Parse Object Pairs function identifies all keys that are equal at the same level
# and calls make_unique to rename the keyname
def parse_object_pairs(pairs):
    dct = OrderedDict()
    #dct = dict()
    for key, value in pairs:
        if key in dct:
            key = make_unique(key, dct)
        dct[key] = value
    return dct

# Make Unique function is in charge of renaming keys which have the same name 
def make_unique(key, dct):
    counter = 0
    unique_key = key

    while unique_key in dct:
        counter += 1
        unique_key = '{}_{}'.format(key, counter)
    return unique_key

# Flatten DIct function is in charge of flattening the nested dictionaries
# While flattening, it will rename the keys at the last level by prepending predecesor key names
def flatten_dict(d, prefix='_'):
    def items():
        # A clojure for recursively extracting dict like values
        for key, value in d.items():
            if isinstance(value, dict):
                for sub_key, sub_value in flatten_dict(value).items():
                    # Key name should imply nested origin of the dict,
                    # so we use a default prefix of __ instead of _ or .
                    if key == "_source" or key == "layers":
                        yield sub_key, sub_value
                    else:
                        yield key + prefix + sub_key, sub_value
            else:
                yield key, value
    return dict(items())

# Define the Main Directory containing all uncompressed PCAPS
pcaps_dir='/home/cc/Contagio'
# Create a list of the Sub-directories within the Main Directory
directories = [x[0] for x in os.walk(pcaps_dir)]

# Print list of Sub-directories
print(directories)
uncpx_pcaps_dir = directories

# Define required empty lists
failed_pcaps=[]

# Define variables
mw_id_counter = 0

# Create List of Unique IDs to identify to assign to each Malware PCAP
mw_random_ids = sorted(random.sample(range(10000,20000), 1300))
#mw_id = mw_random_ids[0]

#Create Empty Dataframe
columns = ['mw_id']
df_ = pd.DataFrame(index=[0], columns=columns)

# The following loop will perform the following actions:
# 1) Loop through every Sub-directory
# 2) Loop through every PCAP within each Sub-directory
# 3) Read every PCAP using tshark filtering only frames containing ssl traffic
#    In addtion, the output of all protocol features will be delivered in a nested JSON format
# 4) The extracted dictionary will be parsed using parse_object_pairs and flatten_dict function
for pcap_dir in uncpx_pcaps_dir[0:1]: #uncpx_pcaps_dir[0:1]
    pcap_dir = "/home/cc/Contagio/Exploit_Kits_and_Malware"
    print("\n")
    print(pcap_dir)
    pcap_dir_files = [f for f in listdir(pcap_dir) if isfile(join(pcap_dir, f))]
    print("\n")
    #pcap_dir_files = ["EK_MALWARE_2014-04-29-fake-flash-updater-02_mailware-traffic-analysis.net.pcap"]
    for i_cap in pcap_dir_files:
        # Get a new malware ID
        mw_id = mw_random_ids[mw_id_counter]
        mw_id_counter = mw_id_counter + 1
        # Print PCAP
        print(i_cap)
        command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-Y", "ssl", "-T", "fields", "-e", "frame.number"]
        #command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-Y", "ssl", "-T", "json"]
        #command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-2", "-R", "ssl", "-Y", "frame.number==1", "-T", "json" ]
        process = Popen(command, universal_newlines=True, stdout=PIPE, stderr=PIPE)
        output, err = process.communicate(b"input data that is passed to subprocess' stdin")
        rc = process.returncode
        # Print Raw Dictionary
        print("=== RAW Dictionary ===")
        print(output)
        print(err)
        #packet_numbers = [int(x) for x in output[:-1].split('\n')]
        packet_numbers = list(map(int, re.findall(r'\d+', output)))
        for p_num in packet_numbers:
            command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-Y", "frame.number=={}".format(p_num), "-T", "json" ]
            process2 = Popen(command, universal_newlines=True, stdout=PIPE, stderr=PIPE)
            output2, err2 = process2.communicate(b"input data that is passed to subprocess' stdin")
            # Print Json Load Dictionary
            #print("=== JSON Load Dictionary ===")
            #json_output = json.loads(output[2:-4])#output[2:-4]
            #pprint(json_output)
        
            # Parse received dictionary
            #"""
            decoder = JSONDecoder(object_pairs_hook=parse_object_pairs)
            user_dict = decoder.decode(output2[2:-4])
            user_dict_final = flatten_dict(user_dict)
            user_dict_final["mw_id"] = mw_id
            #print("\n\n Flatten Dictionary")
            #print("Number of Keys %s" %len(user_dict_final.keys()))
            #pprint(user_dict_final)
            #output = check_output(["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap)], shell=True)
            #print(output)
            #"""

            # Create Dataframe
            df = pd.DataFrame(user_dict_final, index=[1])
            #df_ = pd.concat([df_,df])
            df_ = df_.append([df], ignore_index=True)
output_2 = {}
#extract(output,output_2)

print("\nFailed PCAPS")
print(failed_pcaps)


['/home/cc/Contagio', '/home/cc/Contagio/Exploit_Kits_and_Malware', '/home/cc/Contagio/Exploit_Kits_and_Malware/EK_MALWARE_2014-08-13-sandbox-analysis-pcap-files_mailware-traffic-analysis.net', '/home/cc/Contagio/Exploit_Kits_and_Malware/EK_MALWARE_2015-01-08-pcap-files-associated-with-malware-from-82.244.160.22_mailware-traffic-analysis.net', '/home/cc/Contagio/Exploits_and_Scans', '/home/cc/Contagio/APT', '/home/cc/Contagio/GENERAL_DEFCON_17', '/home/cc/Contagio/General', '/home/cc/Contagio/CRIME', '/home/cc/Contagio/CRIME/tbotpcaps']


/home/cc/Contagio/Exploit_Kits_and_Malware


EK_MALWARE_2014-03-09-callback-traffic-from-physical-host_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-08-29-phishing-malware-01-sandbox-analysis_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-07-19-FlashPack-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-01-07-neutrino-exploit-traffic_mailware-traffic-analys

=== RAW Dictionary ===


EK_MALWARE_2014-05-22-Fiesta-EK-traffic-01_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_Sandbox-analysis-dafee9aa102b64a21c15af6208537dc0_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-06-11-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-07-28-Angler-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-07-15-Magnitude-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_UpdateFlashPlayer_4caf296.exe-malwr.com-analysis_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-09-22-phishing-malware-download_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-10-26-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-06-17-Magnitude-EK-traffic-second-run_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MAL

=== RAW Dictionary ===


EK_MALWARE_2013-11-29-successful-infection_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-08-01-phishing-email-follow-up-malware-sandbox-analysis_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-08-22-unknown-exploit-kit-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-05-20-Rig-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-04-22-Angler-EK-traffic-01_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-07-11-phishing-malware-sandbox-analysis_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-12-11-Asprox-malware-run-on-a-VM_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-08-15-CryptoWall-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===
68
70
71
73
75
78
80
83
84
88
90
92
96
100
102
103
105
106
108
109
111
121
123
125
126


EK_MALWARE_2014-1

=== RAW Dictionary ===


EK_MALWARE_2014-07-20-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-12-01-Neutrino-EK-malware-payload-malwr.com-analysis_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-06-21-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-06-25-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===
2487
2491
2493
2496
2498
2500
2503
2505
2507
2508
2510
2512
2514
2515
2517
2518
2520
2523
2530
2534
2538
2542
2563
2571
2572
2574
2594
2596
2597
2599
2600
2602
2605
2609
2611
2615
2675
2686
2709
2715
2719
2720
2737
2739
2741
2746
2747
2750
2753
2754
2758
2761
2768
2772
2779
2782
2783
2785
2786
2787
2789
2795
2797
2798
2801
2804
2808
2817
2819
2821
2823
2830
2831
2833
2835
2837
2838
2845
2846
2848
2849
2853
2857
2861
2862
2864
2872
2874
2875
2880
2882
2886
2890
2892
2893
2897
2899
2900
2901
2903
2905
2906
2910
2915
2916
2920
2921
2923
2928
29

=== RAW Dictionary ===
529
534
539
547
549
550
552
554
555
557
566
574
576
577
579
580
582
583
585
586
588
605
611
631
651
670
693
694
735
736
969
1001
1099
1141
1148
1171
1180
1186
1187
1189
1190
1192
1194
1195
1201
1219
1220
1224
1239
1241
1245
1264
1279
1283
1287
1293
1299
1305
1306
1308
1310
1311
1313
1314
1316
1317
1319
1320
1335
1337
1339
1340
1342
1343
1345
1346
1348
1349
1351
1357
1365
1409
1492
1499
1504
1505
1507
1508
1514
1516
1533
1549
1550
1551
1553
1555
1556
1573
1596
1614
1632
1648
1661
1662
1668
1687
1703
1719
1736
1751
1768
1784
1800
1818
1833
1849
1865
1881
1892
1893
1904
1921
1936
1952
1969
1988
2002
2020
2038
2055
2073
2091
2109
2125
2142
2156
2171
2191
2205
2218
2233
2252
2270
2287
2303
2320
2337
2352
2369
2386
2403
2421
2438
2455
2473
2491
2502
2518
2521
2535
2544
2547
2548
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2562
2563
2564
2565
2567
2571
2572
2573
2574
2575
2576
2577
2578
2579
2580
2584
2585
2587
2588
2590
2591
2593
2594
2596
2598
2599
2602
2603
260

...

EK_MALWARE_2014-06-23-phishing-malware-sandbox-analysis_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2015-02-11-Windigo-Group-Nuclear-EK-traffic-example-01_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-04-12-FlashPack-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2015-02-09-Angler-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-10-09-Mangitude-EK-payload-5-of-6-malwr.com-analysis_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-02-03-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-09-08-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===
395
396
398


EK_MALWARE_2014-06-14-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-03-09-Fiesta-EK-traffic-failed-attempt_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===

=== RAW Dictionary ===
111
113
119
120
122
123
125
143
144
161
180
198
218
234
243
245
246
247
290
293
295
296
297
299
300
302
303
305
306
307
309
311
312
313
314
316
317
318
319
321
322
324
325
327
328
330
331
333
334
336
337
338


EK_MALWARE_2014-10-02-Angler-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-02-09-Neutrino-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-11-18-Sweet-Orange-EK-malware-payload-from-threatglass-pcap_mailware-traffic-analysis.net.jpg
=== RAW Dictionary ===


EK_MALWARE_2014-11-16-traffic-analysis-exercise_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===
11
12
38
88
111
122
700
716
719
720
722
724
725
727
728
729
730
731
732
733
736
737
746
747
750
753
754
756
758
760
762
763
765
766
768
769
771
772
773
774
775
776
777
778
779
781
783
784
786
787
789
790
792
793
795
796
797
798
801
802
804
805
807
810
811
812
813
815
816
817
818
819
820
821
824
825
826
828
829
831
832
834


EK_MALWARE_2014-03-17-Zuponcic-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===
98
101
102
104
105
107
108
110
111
113
114
116
117
119
120
121


EK_MALWARE_2014-06-12-Sweet-Orange-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===

tshark: The file "/home/cc/Contagio/Exploit_Kits_and_Malware/EK_MALWARE_2014-06-12-Sweet-Orange-EK-traffic_mailware-traffic-analysis.net.pcap" appears to have been cut short in the middle of a packet.

EK_MALWARE_2015-01-12-malwr.com-analysis-of-Sweet-Orange-EK-malware-payload_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-02-11-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-09-21-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===
1146
1147
1149


EK_MALWARE_2014-07-26-Rig-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===
803
806
807
809
817
819
821
822
824
826
844
845
863
865
882
900
918
936
954
972
9

EK_MALWARE_2014-02-28-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-06-26-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-04-22-Angler-EK-traffic-02_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2014-12-16-phishing-malware-run-on-VM_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===
17
19
26
28
30
31
33
36
45
47
48
50
51
53
55
64
66
67
69
70
72
75
84
86
87
89
90
92
94
103
105
106
108
109
111
113
122
124
125
127
128
130
133
142
144
145
147
148
150
152
162
164
165
167
168
170
172
181
183
184
186
187
189
191
201
203
204
206
207
209
210
221
223
224
226
227
229
231
240
242
243
245
246
248
251
260
262
263
265
266
268
271
280
282
283
285
286
288
290
299
301
302
304
305
307
310
319
321
322
324
325
327
330
339
341
342
344
345
347
349
358
360
361
363
364
366
369
378
380
381
383
384
386
388
398
400
401
403
404
406
408
427
429
430
432
433
435
438
447
449
450
452
453
455
457

EK_MALWARE_2014-05-07-Angler-EK-traffic_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===


EK_MALWARE_2015-02-03-chaintor-sample-run-on-a-physical-host_mailware-traffic-analysis.net.pcap
=== RAW Dictionary ===
6
8
12
13
15
16
17
18
25
26
29
30
32
33
98
99
100
105
115
117
121
122
124
126
127
129
134
138
140
141
143
144
145
146
147
152
159
161



Failed PCAPS
[]


In [50]:
# Create list of discovered features
total_features = list(df_.columns.values)
print(len(total_features))
pprint(total_features)


4035
['_index',
 '_score',
 '_type',
 '_ws.malformed__ws.expert__ws.expert.group',
 '_ws.malformed__ws.expert__ws.expert.message',
 '_ws.malformed__ws.expert__ws.expert.severity',
 '_ws.malformed__ws.expert__ws.malformed.expert',
 '_ws.malformed__ws.malformed',
 '_ws.short',
 'data_data.data',
 'data_data.len',
 'eth_eth.dst',
 'eth_eth.dst_tree_eth.addr',
 'eth_eth.dst_tree_eth.addr_resolved',
 'eth_eth.dst_tree_eth.dst_resolved',
 'eth_eth.dst_tree_eth.ig',
 'eth_eth.dst_tree_eth.lg',
 'eth_eth.padding',
 'eth_eth.src',
 'eth_eth.src_tree_eth.addr',
 'eth_eth.src_tree_eth.addr_resolved',
 'eth_eth.src_tree_eth.ig',
 'eth_eth.src_tree_eth.lg',
 'eth_eth.src_tree_eth.src_resolved',
 'eth_eth.type',
 'frame_frame.cap_len',
 'frame_frame.encap_type',
 'frame_frame.ignored',
 'frame_frame.len',
 'frame_frame.marked',
 'frame_frame.number',
 'frame_frame.offset_shift',
 'frame_frame.protocols',
 'frame_frame.time',
 'frame_frame.time_delta',
 'frame_frame.time_delta_displayed',
 'frame_fra

 'signature_algorithms_ssl.handshake.sig_hash_algs_ssl.handshake.sig_hash_alg_tree_6_ssl.handshake.sig_hash_sig',
 'ssl_ssl.record_ssl.handshake_Extension: '
 'signature_algorithms_ssl.handshake.sig_hash_algs_ssl.handshake.sig_hash_alg_tree_7_ssl.handshake.sig_hash_hash',
 'ssl_ssl.record_ssl.handshake_Extension: '
 'signature_algorithms_ssl.handshake.sig_hash_algs_ssl.handshake.sig_hash_alg_tree_7_ssl.handshake.sig_hash_sig',
 'ssl_ssl.record_ssl.handshake_Extension: '
 'signature_algorithms_ssl.handshake.sig_hash_algs_ssl.handshake.sig_hash_alg_tree_8_ssl.handshake.sig_hash_hash',
 'ssl_ssl.record_ssl.handshake_Extension: '
 'signature_algorithms_ssl.handshake.sig_hash_algs_ssl.handshake.sig_hash_alg_tree_8_ssl.handshake.sig_hash_sig',
 'ssl_ssl.record_ssl.handshake_Extension: '
 'signature_algorithms_ssl.handshake.sig_hash_algs_ssl.handshake.sig_hash_alg_tree_9_ssl.handshake.sig_hash_hash',
 'ssl_ssl.record_ssl.handshake_Extension: '
 'signature_algorithms_ssl.handshake.sig_hash_alg

 'ssl_ssl.record_ssl.handshake_ssl.handshake.certificates_ssl.handshake.certificate_tree_x509af.signedCertificate_element_x509af.extensions_tree_x509af.Extension_element_3_x509ce.CRLDistPointsSyntax_tree_x509ce.DistributionPoint_element_x509ce.distributionPoint_tree_x509ce.fullName',
 'ssl_ssl.record_ssl.handshake_ssl.handshake.certificates_ssl.handshake.certificate_tree_x509af.signedCertificate_element_x509af.extensions_tree_x509af.Extension_element_3_x509ce.CRLDistPointsSyntax_tree_x509ce.DistributionPoint_element_x509ce.distributionPoint_tree_x509ce.fullName_tree_x509ce.GeneralName',
 'ssl_ssl.record_ssl.handshake_ssl.handshake.certificates_ssl.handshake.certificate_tree_x509af.signedCertificate_element_x509af.extensions_tree_x509af.Extension_element_3_x509ce.CRLDistPointsSyntax_tree_x509ce.DistributionPoint_element_x509ce.distributionPoint_tree_x509ce.fullName_tree_x509ce.GeneralName_tree_x509ce.uniformResourceIdentifier',
 'ssl_ssl.record_ssl.handshake_ssl.handshake.certificates_s

In [58]:
# Create a dictionary matching Malware IDs with the Malware's Name
print(mw_random_ids[0:10])
mw_id_cntr_2 = 0
mw_id_dict = {}
print(i_cap)
for i_cap2 in pcap_dir_files:
    mw_id_dict[mw_random_ids[mw_id_cntr_2]] = i_cap2
    mw_id_cntr_2 = mw_id_cntr_2 +1
pprint(mw_id_dict)
    
    

[10002, 10004, 10011, 10022, 10034, 10035, 10036, 10037, 10038, 10044]
EK_MALWARE_2015-02-03-chaintor-sample-run-on-a-physical-host_mailware-traffic-analysis.net.pcap
{10002: 'EK_MALWARE_2014-03-09-callback-traffic-from-physical-host_mailware-traffic-analysis.net.pcap',
 10004: 'EK_MALWARE_2014-08-29-phishing-malware-01-sandbox-analysis_mailware-traffic-analysis.net.pcap',
 10011: 'EK_MALWARE_2014-07-19-FlashPack-EK-traffic_mailware-traffic-analysis.net.pcap',
 10022: 'EK_MALWARE_2014-01-07-neutrino-exploit-traffic_mailware-traffic-analysis.net.pcap',
 10034: 'EK_MALWARE_2014-11-18-Sweet-Orange-EK-traffic_mailware-traffic-analysis.net.pcap',
 10035: 'EK_MALWARE_2014-04-29-fake-flash-updater-02_mailware-traffic-analysis.net.pcap',
 10036: 'EK_MALWARE_2014-03-27-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap',
 10037: 'EK_MALWARE_2014-05-11-FlashPack-EK-traffic_mailware-traffic-analysis.net.pcap',
 10038: 'EK_MALWARE_2015-01-23-Nuclear-EK-and-vawtrak-traffic_mailware-traffic-analy

In [69]:
# Create mw_id_series
mw_id_series = pd.Series(mw_id_dict)
# Save mw_id_series
mw_id_series.to_pickle('mw_id_series.p')
# Save SSL df (df_)
df_.to_pickle('mw_ssl_dataframe.p')
# Save SSL features list
ssl_features_file = open('ssl_features.txt', 'w')
for item in total_features:
    ssl_features_file.write("%s\n" % item)

# Sources:

## Tshark
https://ask.wireshark.org/questions/58343/2-pass-filter-in-wiresharktshark
https://hackertarget.com/tshark-tutorial-and-filter-examples/
https://www.wireshark.org/docs/wsug_html_chunked/AppToolstshark.html
http://www.thegeekstuff.com/2012/07/wireshark-filter/?ref=driverlayer

## Pandas and JSON
https://www.dataquest.io/blog/python-json-tutorial/
https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
https://pandas.pydata.org/pandas-docs/stable/merging.html
https://stackoverflow.com/questions/23019119/converting-multilevel-nested-dictionaries-to-pandas-dataframe
http://pbpython.com/pandas-list-dict.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html
https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-then-filling-it
https://pandas.pydata.org/pandas-docs/stable/options.html
http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/
https://www.digitalocean.com/community/tutorials/an-introduction-to-the-pandas-package-and-its-data-structures-in-python-3

## Unique Dictionary Keys
https://stackoverflow.com/questions/29321677/python-json-parser-allow-duplicate-keys
https://stackoverflow.com/questions/36086470/parsing-python-json-with-multiple-same-strings-with-different-values
https://stackoverflow.com/questions/29321677/python-json-parser-allow-duplicate-keys/29323197

## Flattening Dictionaries
https://stackoverflow.com/questions/12507206/python-recommended-way-to-walk-complex-dictionary-structures-imported-from-json
https://stackoverflow.com/questions/6027558/flatten-nested-python-dictionaries-compressing-keys

## Zip Files
https://unix.stackexchange.com/questions/270555/how-to-get-number-of-files-in-a-7z